# BibRec: Re-Training Random Forest Model

### Features: Country, State, Age, Year-of-Publication, Publisher

In [13]:
from bibrec.server.data_exploration import get_normalized_data, hot_encode_country
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
import seaborn as sns
import matplotlib.pyplot as plt

books, users, ratings = get_normalized_data(books_path='data/BX-Books.csv', users_path='data/BX-Users.csv', ratings_path='data/BX-Book-Ratings.csv')


/Dev/repos/FH-SBG/MMT1/RES/recommender-systems/wt-rf/bibrec/server/data_exploration.py:6: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv(path, sep=";", encoding="latin-1")


With NaN values 34.862889904962536
used mean values 34.89615301832869


/Dev/repos/FH-SBG/MMT1/RES/recommender-systems/wt-rf/bibrec/server/data_exploration.py:37: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  location_seperated = users.location.str.split(',', 2, expand=True)


In [14]:
# limit data volume
df_ratings = ratings.groupby('isbn13').user_id.count().sort_values(ascending=False)
df_ratings = ratings[1000:2000]
df_ratings = df_ratings.reset_index()


In [15]:
# RF Features: Country, State, Age, Year-of-Publication, Publisher
tmp_users = users.filter(regex="user_id|age|country_|state_", axis=1)
tmp_books = books.filter(regex="isbn13|normalized_year_of_publication|publisher_", axis=1)
# df = df_ratings.filter(regex="isbn13|user_id|normalized_rating", axis=1)
df = df_ratings.filter(regex="isbn13|user_id|book_rating", axis=1)
df = df.merge(tmp_users)
df = df.merge(tmp_books)

In [16]:
df_ratings

,index,user_id,isbn,book_rating,isbn13,age,city,state,country,user_mean,user_count,normalized_rating
0,1000,277784,0440802911,8,978044080297,17.000000,baton rouge,louisiana,usa,8.0,1.0,0.0
1,1001,277788,0440407524,8,9780440407522,17.000000,baton rouge,louisiana,usa,8.0,1.0,0.0
2,1002,277794,0440967694,8,9780440967699,47.843881,edmonton,alberta,canada,8.0,1.0,0.0
3,1003,277795,0064406601,5,9780064406604,17.000000,zachary,louisiana,usa,5.0,1.0,0.0
4,1004,277797,0440228441,8,9780440228448,17.000000,baton rouge,louisiana,usa,8.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
995,1995,278663,0451182014,3,9780451182012,27.716546,arlington,texas,usa,5.8,5.0,-2.8
996,1996,278668,8408022938,5,9788408022930,48.000000,madrid,madrid,spain,5.0,5.0,0.0
997,1997,278668,8408044079,5,9788408044079,48.000000,madrid,madrid,spain,5.0,5.0,0.0
998,1998,278668,8423322912,5,9788423322916,48.000000,madrid,madrid,spain,5.0,5.0,0.0


In [17]:
# Features
X = df.drop(['user_id', 'isbn13', 'book_rating'], axis=1)
# Prediction
Y = df['book_rating']


In [18]:
X

,age,country_usa,country_canada,country_united_kingdom,country_germany,country_spain,country_australia,country_italy,country_france,country_portugal,...,publisher_silhouette,publisher_pocket,publisher_ballantine_books,publisher_bantam_books,publisher_scholastic,publisher_simon_&amp;_schuster,publisher_penguin_books,publisher_berkley_publishing_group,publisher_warner_books,publisher_other
0,17.000000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,17.000000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,47.843881,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,17.000000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,17.000000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1010,27.716546,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1011,48.000000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1012,48.000000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1013,48.000000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [19]:
Y

0       8
1       8
2       8
3       5
4       8
       ..
1010    3
1011    5
1012    5
1013    5
1014    5
Name: book_rating, Length: 1015, dtype: int64

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.30, random_state=7)

In [21]:
import pickle

# Load the model from the file
with open("random_forest_classifier5.pkl", "rb") as file:
    rfc = pickle.load(file)

In [22]:
rfc.fit(X_train, y_train)

RandomForestClassifier(min_weight_fraction_leaf=0, n_jobs=3, random_state=1)

In [23]:
rfc_pred = rfc.predict(X_test)
print(classification_report(y_test, rfc_pred))
print(accuracy_score(y_test, rfc_pred) * 100)

              precision    recall  f1-score   support

           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         2
           3       0.00      0.00      0.00         7
           4       0.00      0.00      0.00        16
           5       0.37      0.26      0.30        39
           6       0.19      0.17      0.18        30
           7       0.23      0.25      0.24        65
           8       0.22      0.28      0.25        69
           9       0.20      0.23      0.22        44
          10       0.15      0.15      0.15        33

    accuracy                           0.21       305
   macro avg       0.14      0.13      0.13       305
weighted avg       0.21      0.21      0.21       305

21.311475409836063


/home/fab/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/fab/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/fab/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
